## Scraping y recolección de datos de la web de Facua

In [ ]:
from bs4 import BeautifulSoup
import requests
import numpy as np
from time import sleep
import json
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings('ignore')

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd


# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

El primer paso de este proyecto es obtener para cada uno de los supermercados de la web de facua el precio a lo largo del tiempo de tres tipo de productos: aceite de giralos, aceite de oliva y leche. Para ello usaremos Beautiful soup y selenium: 

In [59]:
driver = webdriver.Chrome()
url="https://super.facua.org/"
driver.get(url)
driver.maximize_window()
sleep(1)

# Diccionario donde por super y categoría almacenaremos los links de las páginas de los productos
diccionario_url = {
    "mercadona":{},
    "carrefour":{},
    "eroski":{},
    "dia":{},
    "hipercor":{},
    "alcampo":{},
}

# Itero por supermercado
for i in range(1,7):
    sleep(1)
    driver.find_element("css selector", f"body > section:nth-child(4) > div > div.row.gx-4.gx-lg-6.row-cols-2.row-cols-md-2.row-cols-xl-6.justify-content-center > div:nth-child({i}) > div > div.card-footer.p-4.pt-0.border-top-0.bg-transparent > div > a").click()
    sleep(2)

    aux_elements=driver.find_elements("css selector", "body > section:nth-child(4) > div > div.row.gx-4.gx-lg-5.row-cols-2.row-cols-md-3.row-cols-xl-4.justify-content-center")
    aux = [element.text for element in aux_elements]
    lista_productos = aux[0].split("\n")[::2]
    
    j=1
    for producto in lista_productos:
        
        # -------------Aceite de girasol, aceite de oliva, leche------------------
        driver.find_element("css selector", f"body > section:nth-child(4) > div > div.row.gx-4.gx-lg-5.row-cols-2.row-cols-md-3.row-cols-xl-4.justify-content-center > div:nth-child({j}) > div > div.card-footer.p-4.pt-0.border-top-0.bg-transparent > div > a").click()
        sleep(2)

        #Saco los link de los productos de aceite de girasol
        html_table_page = driver.page_source
        sopa = BeautifulSoup(html_table_page, "html.parser")
        productos = sopa.findAll("div", {"class":"card-footer p-4 pt-0 border-top-0 bg-transparent"})
        lista_links = []
        for prod in productos:
            link = prod.find("a", {"class":"btn-unirme btn-verde inline-block inline-block bg-primary border-primary font-semibold rounded-full"}).get("href")
            lista_links.append(link)
        
        #Sacamos la clave correspondiente para el diccionario
        clave = list(diccionario_url.keys())[i-1]
        diccionario_url[clave][producto]=lista_links

        driver.back()
        sleep(1)
        j+=1

    driver.back()
    sleep(2)
    
driver.quit()

In [64]:
with open("../datos/diccionario_links.json", "w") as archivo:
    json.dump(diccionario_url, archivo)

Cargamos el diccionario

In [68]:
with open("../datos/diccionario_links.json", "r") as archivo:
    diccionario_url = json.load(archivo)

Creamos un dataframe para cada producto y supermercado

In [135]:
# iteramos para cada super
for clave, valor in tqdm(diccionario_url.items()):
    #Iteramos para cada producto
    for producto in lista_productos:
        links = diccionario_url[clave][producto]
        df_producto = pd.DataFrame()
        for link in links:
            # Meto un try excepy porque hay algunos links que no son de productos sino de una categoría
            try:
                url = link
                contenido = requests.get(url)
                contenido.status_code
                sopa = BeautifulSoup(contenido.content, "html.parser")

                tabla = sopa.find("table", {"class":"table table-striped table-responsive text-center"})

                enunciados = [enunciado.getText() for enunciado in tabla.findAll("th")]

                filas_tabla=[]
                for fila in tabla.findAll("tr"):
                    fila_sucia= fila.findAll("td")
                    aux=[elem.text for elem in fila_sucia]
                    filas_tabla.append(aux)
                
                df= pd.DataFrame(filas_tabla[1:], columns=enunciados)
                df["supermercado"] = clave
                df["producto"] = producto
                df["nombre_producto"] = url.split("/")[-2].replace("-"," ")
                df_producto = pd.concat([df_producto, df])
                
            except:
                pass

        df_producto.to_csv(f"../datos/df_{clave}_{producto}.csv")

100%|██████████| 6/6 [04:30<00:00, 45.09s/it]


## Creación y limpieza del dataframe

In [2]:
os.chdir("../datos")
datos = os.listdir()[:-1]
df=pd.DataFrame()

for i in datos:
    df_aux = pd.read_csv(f"../datos/{i}")
    # print(i)
    df = pd.concat([df, df_aux])
df

,Unnamed: 0,Día,Precio (€),Variación,supermercado,producto,nombre_producto
0,0,30/06/2024,"5,88",=,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...
1,1,01/07/2024,"5,88",=,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...
2,2,12/07/2024,"5,88",=,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...
3,3,13/07/2024,"5,88",=,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...
4,4,14/07/2024,"5,88",=,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...
...,...,...,...,...,...,...,...
2860,98,21/10/2024,"9,36",=,mercadona,Leche,leche semidesnatada sin lactosa hacendado 9 l
2861,99,22/10/2024,"9,36",=,mercadona,Leche,leche semidesnatada sin lactosa hacendado 9 l
2862,100,23/10/2024,"9,36",=,mercadona,Leche,leche semidesnatada sin lactosa hacendado 9 l
2863,101,24/10/2024,"9,36",=,mercadona,Leche,leche semidesnatada sin lactosa hacendado 9 l


In [3]:
df.drop(columns=["Unnamed: 0"], inplace=True)

In [4]:
df = df.reindex(columns=['supermercado', 'producto','nombre_producto', 'Día', 'Precio (€)', 'Variación'])
df

,supermercado,producto,nombre_producto,Día,Precio (€),Variación
0,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...,30/06/2024,"5,88",=
1,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...,01/07/2024,"5,88",=
2,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...,12/07/2024,"5,88",=
3,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...,13/07/2024,"5,88",=
4,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...,14/07/2024,"5,88",=
...,...,...,...,...,...,...
2860,mercadona,Leche,leche semidesnatada sin lactosa hacendado 9 l,21/10/2024,"9,36",=
2861,mercadona,Leche,leche semidesnatada sin lactosa hacendado 9 l,22/10/2024,"9,36",=
2862,mercadona,Leche,leche semidesnatada sin lactosa hacendado 9 l,23/10/2024,"9,36",=
2863,mercadona,Leche,leche semidesnatada sin lactosa hacendado 9 l,24/10/2024,"9,36",=


Cambiamos la columna de variación para hacer dos columnas, una indicando si ha habido un aumento o una disminucion y otra con el porcentaje.

In [5]:
df["Variación"] = df["Variación"].apply(lambda x: np.nan if x=='=' else x)

In [6]:
# Pasamos de una columna a dos
df_aux = df["Variación"].str.split(" ", expand=True)
df = pd.concat([df,df_aux], axis=1)

In [7]:
# Eliminamos la columna y renombramos
df.drop(columns=["Variación"], inplace=True)
df.rename(columns={0: 'incremento', 1:"porcentaje"}, inplace=True)

In [8]:
len(df["nombre_producto"].unique())

1597

En incremento dejamos solo si es aumento o decremento y en porcentaje solo el número:

In [9]:
df["incremento"]=df["incremento"].apply(lambda x: x[0] if type(x)==str else x)
df["incremento"]=df["incremento"].map({'+':'aumento', '-':'decremento'})
df["porcentaje"]=df["porcentaje"].str.replace(r"[()%]", "", regex=True)

Ahora intentaremos sacar una subcategoría a partir del nombre

In [10]:
df["nombre_producto"]=df["nombre_producto"].str.lower()
df["producto"]=df["producto"].str.lower()


In [11]:
def obtener_subcategoria(row):
    if row['producto'] == 'aceite de oliva':
        if 'virgen extra' in row['nombre_producto']:
            return 'virgen extra'
        elif 'virgen' in row['nombre_producto']:
            return 'virgen'
        else:
            return np.nan
    elif row['producto'] == 'leche':
        if 'entera' in row['nombre_producto']:
            return 'entera'
        elif 'semi' in row['nombre_producto'] or 'semidesnatada' in row['nombre_producto']:
            return 'semidesnatada'
        elif 'desnatada' in row['nombre_producto']:
            return 'desnatada'
        else:
            return np.nan
    elif row['nombre_producto'] == 'aceite de girasol':
        return np.nan
    else:
        return np.nan

# Aplicar la función al DataFrame para crear la columna subcategoria
df['subcategoria'] = df.apply(obtener_subcategoria, axis=1)


In [15]:
df=df.reindex(columns=['supermercado', 'producto', 'nombre_producto', 'subcategoria', 'Día', 'Precio (€)',
       'incremento', 'porcentaje'])

In [18]:
df.to_csv("../datos/df_supermercados.csv")